# 🚀 Ai_wolf (Wolf_V5) 專案 Colab 快速啟動器 (超穩健版)

本 Notebook 用於在 Google Colaboratory 環境中一鍵部署和啟動 Ai_wolf (Wolf_V5) Streamlit 應用程式。

**特點**:
*   **強制環境清理**：每次執行都會清理舊的專案檔案，確保從乾淨的狀態開始。
*   **自動化依賴安裝**：從 `requirements.txt` 安裝必要的 Python 套件。
*   **最新程式碼部署**：自動從 GitHub 克隆最新的專案程式碼。
*   **直接錯誤顯示**：此版本腳本會直接捕獲並顯示 Streamlit 應用程式的所有輸出（包括標準輸出和錯誤輸出）。如果應用程式在啟動時崩潰，相關的 Python 錯誤追蹤 (Traceback) 會直接顯示在 Colab 的儲存格輸出中，方便快速定位問題。
*   **公開網址提供**：透過 Colab 內建代理或 Cloudflared 提供一個公開的 URL 來訪問應用程式。

**首次使用設定**:
1.  **API 金鑰**:
    *   在 Colab 的左側邊欄找到鑰匙圖示 (Secrets)。
    *   新增兩個 Secret：
        *   `GOOGLE_API_KEY`：填入您的 Google Gemini API 金鑰。
        *   `FRED_API_KEY`：填入您的 FRED API 金鑰。
    *   確保 "Notebook access" 已啟用。應用程式 (`app.py`) 會優先從這裡讀取金鑰。
2.  **Google Drive 授權**:
    *   執行此 Notebook 時，會提示您授權 Google Drive 存取。這是為了將專案檔案和未來可能的資料（如日誌）儲存在您的 Drive 中，以便持久化。
    *   專案將被下載到 `/content/drive/MyDrive/wolfAI`。
    *   應用程式日誌將寫入 `/content/drive/MyDrive/Wolf_Data/logs/streamlit.log` (已更新)。

**執行步驟**:
1.  點擊下方 Code Cell 左側的「執行」按鈕。
2.  耐心等待所有階段完成（環境設定、套件安裝、專案下載、服務啟動）。
3.  成功後，會顯示一個**綠色的按鈕**，點擊即可在新分頁中打開 Ai_wolf 應用程式。
4.  Colab 儲存格的輸出會持續顯示來自 Streamlit 應用程式的即時日誌和任何錯誤訊息。

**注意**:
*   如果遇到任何問題，請仔細檢查 Colab 儲存格中的輸出訊息，特別是任何包含 "Error" 或 "Traceback" 的部分。
*   要停止應用程式，可以點擊 Colab 儲存格執行按鈕旁邊的「停止」按鈕，或在儲存格運行時按 `Ctrl+C` (或 `Cmd+C`)。

In [ ]:
# === Wolf_V5 專案：環境設置、部署、啟動與即時監控 (單一儲存格) ===
# 此版本直接捕獲 Streamlit 的所有輸出，能立即顯示啟動錯誤。

import os
import subprocess
import time
import threading
import sys # Added for sys.executable
from IPython.display import display, HTML, clear_output
# google.colab.drive 和 google.colab.output 只在 Colab 中可用
try:
    from google.colab import drive
    from google.colab.output import eval_js
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    # 如果需要在本地模擬這些行為，可以在這裡定義 mock 函數或類別
    def mock_drive_mount(path, force_remount=False): print(f"Mock: Drive mount requested for {path}")
    drive = type('Drive', (object,), {'mount': mock_drive_mount})()
    def mock_eval_js(code): print(f"Mock: eval_js called with {code}"); return "http://localhost:8501" # 返回一個模擬 URL
    eval_js = mock_eval_js
    print("非 Colab 環境，部分 Colab 特定功能將被模擬或跳過。")


# --- 階段一：環境設置與專案部署 ---
print("--- 階段一：環境設置與專案部署 ---")
print("\n[1/5] 正在安裝必要的 Python 套件...")
get_ipython().system('pip install streamlit google-generativeai yfinance pandas fredapi requests -q')
print("✅ 套件安裝完成！")

GDRIVE_PROJECT_DIR = "/content/drive/MyDrive/wolfAI" 
LOCAL_PROJECT_DIR = "/content/wolfAI" 
PROJECT_DIR_TO_USE = None 

if IN_COLAB:
    print("\n[2/5] 正在嘗試掛載 Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 掛載成功！")
        PROJECT_DIR_TO_USE = GDRIVE_PROJECT_DIR

        WOLF_DATA_BASE_GDRIVE = "/content/drive/MyDrive/Wolf_Data"
        print(f"    檢查/創建 Wolf_Data 基礎目錄: {WOLF_DATA_BASE_GDRIVE}")
        if not os.path.exists(WOLF_DATA_BASE_GDRIVE):
            os.makedirs(WOLF_DATA_BASE_GDRIVE, exist_ok=True)
            print(f"    ✅ '{WOLF_DATA_BASE_GDRIVE}' 已創建。")
        else:
            print(f"    ➡️ '{WOLF_DATA_BASE_GDRIVE}' 已存在。")

        sub_dirs_to_create = ["source_documents/masters", "source_documents/shan_jia_lang", "weekly_reports", "logs"]
        for sub_dir in sub_dirs_to_create:
            full_sub_dir_path = os.path.join(WOLF_DATA_BASE_GDRIVE, sub_dir)
            print(f"    檢查/創建子目錄: {full_sub_dir_path}")
            if not os.path.exists(full_sub_dir_path):
                os.makedirs(full_sub_dir_path, exist_ok=True)
                print(f"    ✅ '{full_sub_dir_path}' 已創建。")
            else:
                print(f"    ➡️ '{full_sub_dir_path}' 已存在。")

    except Exception as e:
        print(f"⚠️ Google Drive 掛載失敗: {e}")
        print("將嘗試使用 Colab 本地儲存空間。如果重新執行，之前在 Drive 中的資料可能不會被使用。")
        PROJECT_DIR_TO_USE = LOCAL_PROJECT_DIR
else:
    print("\n[2/5] 跳過 Google Drive 掛載 (非 Colab 環境)。")
    PROJECT_DIR_TO_USE = LOCAL_PROJECT_DIR 

WOLF_DATA_GDRIVE_BASE_INFO = "/content/drive/MyDrive/Wolf_Data" 
WOLF_DATA_LOCAL_BASE_INFO = "/content/Wolf_Data" 
APP_EXPECTED_LOG_DIR = None 

if PROJECT_DIR_TO_USE == GDRIVE_PROJECT_DIR: 
    APP_EXPECTED_LOG_DIR = os.path.join(WOLF_DATA_GDRIVE_BASE_INFO, "logs")
    if not os.path.exists(WOLF_DATA_GDRIVE_BASE_INFO):
        print(f"    ⚠️ Sanity Check: Wolf_Data base {WOLF_DATA_GDRIVE_BASE_INFO} not found, creating.")
        os.makedirs(WOLF_DATA_GDRIVE_BASE_INFO, exist_ok=True)
else: 
    APP_EXPECTED_LOG_DIR = os.path.join(WOLF_DATA_LOCAL_BASE_INFO, "logs")
    if not os.path.exists(WOLF_DATA_LOCAL_BASE_INFO): 
        os.makedirs(WOLF_DATA_LOCAL_BASE_INFO, exist_ok=True)
        print(f"    ✅ 已確認/建立模擬的本地 Wolf_Data 基礎目錄: {WOLF_DATA_LOCAL_BASE_INFO}")

print(f"\n[3/5] 設定專案路徑: {PROJECT_DIR_TO_USE}")
print(f"    應用程式資料/日誌目錄預期在: {APP_EXPECTED_LOG_DIR} (實際路徑由 app.py 設定)")

if os.path.exists(PROJECT_DIR_TO_USE):
    print(f"    偵測到已存在的專案目錄 '{PROJECT_DIR_TO_USE}'。執行強制清理...")
    try:
        import shutil
        shutil.rmtree(PROJECT_DIR_TO_USE)
        print(f"    ✅ '{PROJECT_DIR_TO_USE}' 已成功刪除。")
    except Exception as e_rm:
        print(f"    ❌ 刪除 '{PROJECT_DIR_TO_USE}' 時發生錯誤: {e_rm}")
        print("       請手動檢查並刪除該資料夾，然後重新執行此儲存格。")
        raise 
else:
    print(f"    '{PROJECT_DIR_TO_USE}' 不存在，無需清理。")

project_parent_dir = os.path.dirname(PROJECT_DIR_TO_USE)
if not os.path.exists(project_parent_dir) and PROJECT_DIR_TO_USE != LOCAL_PROJECT_DIR: 
     os.makedirs(project_parent_dir, exist_ok=True) 

STREAMLIT_APP_PATH = os.path.join(PROJECT_DIR_TO_USE, "app.py")
REQUIREMENTS_PATH = os.path.join(PROJECT_DIR_TO_USE, "requirements.txt")
PORT = 8501

print(f"✅ 已確認/建立專案目錄。") 
print(f"\n[4/5] 正在從 GitHub 獲取最新程式碼...")
GIT_REPO_URL = "https://github.com/hsp1234-web/Ai_wolf.git" 
git_clone_command = ["git", "clone", "--depth", "1", GIT_REPO_URL, PROJECT_DIR_TO_USE]
result = subprocess.run(git_clone_command, capture_output=True, text=True)
if result.returncode == 0:
    print("    ✅ 專案克隆完成！")
else:
    print(f"    ❌ 專案克隆失敗:")
    print(result.stdout)
    print(result.stderr)
    raise SystemExit("Git clone 失敗，終止執行。")

if os.path.isfile(REQUIREMENTS_PATH):
    print("    正在從 requirements.txt 安裝依賴...")
    pip_install_command = ["pip", "install", "-q", "-r", REQUIREMENTS_PATH]
    result_pip = subprocess.run(pip_install_command, capture_output=True, text=True)
    if result_pip.returncode == 0:
        print("    ✅ 依賴安裝完成！")
    else:
        print(f"    ❌ 依賴安裝失敗:")
        print(result_pip.stdout) 
        print(result_pip.stderr)
else:
    print(f"    ⚠️ 警告: 未找到 requirements.txt 於 {REQUIREMENTS_PATH}。跳過依賴安裝步驟。")

print(f"\n[5/5] 檢查主要應用程式檔案...")
if not os.path.isfile(STREAMLIT_APP_PATH):
    raise SystemExit(f"❌ 錯誤：未在 {PROJECT_DIR_TO_USE} 中找到 app.py！")
print(f"✅ 成功找到主要應用程式檔案: {STREAMLIT_APP_PATH}")
print("\n--- 環境設置與專案部署階段完成 ---\n")
time.sleep(2)
if IN_COLAB: clear_output(wait=True)

# --- 階段二：啟動 Streamlit 並直接監控其輸出 ---
print("--- 階段二：啟動 Streamlit 並直接監控其輸出 ---")
streamlit_process = None
monitor_thread = None

# --- Helper Functions for URL Fetching and Display ---
EVAL_JS_MAX_RETRIES = 5
EVAL_JS_RETRY_DELAY = 5
CLOUDFLARED_TIMEOUT_SECONDS = 30

def fetch_url_with_eval_js(port_num, max_retries_eval, retry_delay_eval, eval_js_func, time_module, print_func, in_colab_flag):
    if not in_colab_flag:
        print_func("   eval_js: Not in Colab, skipping this method.")
        return None
    
    local_tunnel_url = None
    print_func("\n🔗 [URL Fetch Method 1: Colab eval_js] 開始嘗試獲取應用程式網址...")
    for attempt in range(max_retries_eval):
        print_func(f"   Attempting to fetch URL via eval_js (try {attempt + 1}/{max_retries_eval})...")
        try:
            proxy_url_eval = eval_js_func(f'google.colab.kernel.proxyPort({port_num})', timeout_sec=10)
            if proxy_url_eval:
                local_tunnel_url = proxy_url_eval
                print_func(f"   SUCCESS: eval_js returned URL: {local_tunnel_url}")
                break
            else:
                print_func(f"   INFO: eval_js attempt {attempt + 1} returned no URL.")
        except NameError: 
            print_func("   WARNING: eval_js attempt NameError (eval_js not defined). Cannot use this method.")
            break 
        except Exception as e:
            print_func(f"   WARNING: eval_js attempt {attempt + 1} failed with error: {e}")
            if 'proxyPort is not defined' in str(e) or 'google.colab.kernel.proxyPort is not a function' in str(e):
                print_func(f"   INFO: Colab Kernel ProxyPort feature seems unavailable at this moment.")
        if attempt < max_retries_eval - 1:
            print_func(f"   Retrying in {retry_delay_eval} seconds...")
            time_module.sleep(retry_delay_eval)
        else:
            print_func(f"   ❌ eval_js: Reached max retries ({max_retries_eval}), failed to get URL.")
    return local_tunnel_url

def fetch_url_with_cloudflared(port_num, timeout_seconds_cf, subprocess_mod, os_mod, threading_mod, time_mod, print_func):
    print_func("\n🔗 [URL Fetch Method 2: cloudflared] Attempting to fetch URL via cloudflared...")
    tunnel_url_cf_holder = [None] 
    cloudflared_process_cf = None 
    try:
        subprocess_mod.run(["cloudflared", "--version"], check=True, capture_output=True)
        print_func("    cloudflared 已安裝。")
    except (subprocess_mod.CalledProcessError, FileNotFoundError):
        print_func("    cloudflared 未安裝或未在 PATH 中。正在嘗試安裝...")
        npm_install_cmd = ["npm", "install", "-g", "cloudflared"]
        npm_result = subprocess_mod.run(npm_install_cmd, capture_output=True, text=True)
        if npm_result.returncode == 0:
            print_func("    ✅ cloudflared 安裝成功 (透過 npm)。")
        else:
            print_func(f"    ❌ cloudflared (npm) 安裝失敗: {npm_result.stderr}")
            return None 

    cloudflared_process_cf = subprocess_mod.Popen(
        ['cloudflared', 'tunnel', '--url', f'http://localhost:{port_num}'],
        stdout=subprocess_mod.PIPE, stderr=subprocess_mod.PIPE, text=True, encoding='utf-8'
    )

    def get_cf_url_from_stderr(stderr_stream, url_holder_list, proc):
        for line in iter(stderr_stream.readline, ''):
            print_func(f"[Cloudflared] {line}", end='', flush=True)
            if ".trycloudflare.com" in line:
                extracted_url = line[line.find("https://"):].split(" ")[0]
                url_holder_list[0] = extracted_url
                print_func(f"    SUCCESS: cloudflared thread found URL: {url_holder_list[0]}")
                break 
            if proc and proc.poll() is not None: # Stop if process died
                break
        stderr_stream.close()

    cf_thread = threading_mod.Thread(target=get_cf_url_from_stderr, args=(cloudflared_process_cf.stderr, tunnel_url_cf_holder, cloudflared_process_cf))
    cf_thread.daemon = True
    cf_thread.start()

    start_time = time_mod.time()
    while not tunnel_url_cf_holder[0] and (time_mod.time() - start_time) < timeout_seconds_cf:
        if cloudflared_process_cf.poll() is not None: 
            print_func("   WARNING: cloudflared process terminated unexpectedly.")
            break
        time_mod.sleep(1)
    
    if not tunnel_url_cf_holder[0]:
        print_func("   WARNING: cloudflared timed out or failed to provide a URL. Check cloudflared logs.")
        if cloudflared_process_cf.poll() is None: 
             try:
                cloudflared_process_cf.terminate()
                cloudflared_process_cf.wait(timeout=5)
             except Exception:
                cloudflared_process_cf.kill()
    else:
        print_func("   INFO: cloudflared setup appears complete (URL may have been found).")
    
    return tunnel_url_cf_holder[0]

SUCCESS_HTML_TEMPLATE_STR = """
    <div style='border: 2px solid #4CAF50; padding: 20px; border-radius: 10px;'>
        <h2 style='color: #2E7D32;'>🎉 應用程式已成功啟動！</h2>
        <p>日誌和錯誤將直接顯示在本儲存格的下方。</p>
        <p><a href='{url}' target='_blank' style='padding:12px 25px; background-color:#4CAF50; color:white; text-decoration:none; border-radius:8px;'>
           🚀 點此開啟 Wolf_V5 應用程式
        </a></p>
        <p style='font-size:0.8em; color:grey;'>如果無法訪問，請檢查下方 Streamlit 和 Cloudflared (如果使用) 的輸出日誌。</p>
    </div>
"""

FAILURE_HTML_TEMPLATE_STR = """
    <div style='border: 2px solid #F44336; padding: 20px; border-radius: 10px; background-color: #fff0f0;'>
        <h2 style='color: #C62828;'>❌ 自動獲取應用程式訪問連結失敗</h2>
        <p style='color:#D32F2F;'>經過多次嘗試 (包括 Colab 內建代理和 Cloudflared)，腳本未能自動獲取到一個可公開訪問的 URL。</p>
        <p style='margin-top:15px;'><strong>可能原因及排查建議：</strong></p>
        <ul style='text-align:left; margin-left: 20px;'>
            <li><strong>Streamlit 服務未成功啟動：</strong>請向上回顧本儲存格的日誌輸出，特別是 <code>[Streamlit]</code> 開頭的行。檢查是否有任何錯誤訊息 (Error, Traceback) 指示 Streamlit 應用程式 (app.py) 啟動失敗。</li>
            <li><strong>Colab 環境問題：</strong>
                <ul>
                    <li>Colab 的 <code>google.colab.kernel.proxyPort</code> 功能可能暫時不可用或遇到問題。</li>
                    <li>如果使用了 Cloudflared，它可能未能正確安裝或啟動 (檢查 <code>[Cloudflared]</code> 開頭的日誌)。</li>
                </ul>
            </li>
            <li><strong>網路限制：</strong>您當前的網路環境或 Colab 的網路狀態可能限制了外部隧道的建立。</li>
        </ul>
        <p style='margin-top:15px;'><strong>您可以嘗試以下手動操作：</strong></p>
        <ul style='text-align:left; margin-left: 20px;'>
            <li><strong>檢查 Streamlit 日誌中的 URL：</strong> 在上方的 <code>[Streamlit]</code> 日誌中，尋找類似 <code>Network URL: http://xxx.xxx.xxx.xxx:xxxx</code> 或 <code>External URL: http://xxxxxxxx.ngrok.io</code> (或其他隧道服務商) 的行。如果 Streamlit 成功啟動並由其自身或某個插件建立了隧道，它通常會打印出來。您可以嘗試手動複製這些 URL 到瀏覽器中訪問。</li>
            <li><strong>查看 Colab Runtime Logs：</strong> 在 Colab 菜單欄選擇「執行階段」(Runtime) -> 「查看執行階段日誌」(View runtime logs)，檢查是否有更底層的錯誤信息。</li>
            <li><strong>重新執行儲存格：</strong> 有時 Colab 環境的臨時問題可以通過重新執行整個儲存格解決。</li>
        </ul>
        <p style='font-size:0.9em; color:gray; margin-top:20px;'>請注意：Streamlit 應用程式本身可能仍在後台運行，即使此處未能生成直接訪問按鈕。</p>
    </div>
"""

def display_result_html(url_to_display, success_template, failure_template, display_func, HTML_class, clear_output_func, in_colab_flag, print_func):
    print_func("\n--- Preparing to display access link/message ---")
    if in_colab_flag: 
        clear_output_func(wait=True)
    if url_to_display:
        display_func(HTML_class(success_template.format(url=url_to_display)))
    else:
        display_func(HTML_class(failure_template))

try:
    cmd_streamlit = [sys.executable, "-m", "streamlit", "run", STREAMLIT_APP_PATH, "--server.port", str(PORT), "--server.headless", "true", "--server.enableCORS", "false", "--server.enableXsrfProtection", "false"]

    streamlit_process = subprocess.Popen(cmd_streamlit, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', cwd=PROJECT_DIR_TO_USE)
    print("✅ Streamlit 啟動指令已送出。")
    print("⏳ 等待伺服器初始化並嘗試獲取連結...")

    def stream_watcher(identifier, stream):
        for line in iter(stream.readline, ''):
            if not line:
                break
            print(f"[{identifier}] {line}", end='', flush=True)
        stream.close()

    monitor_thread = threading.Thread(target=stream_watcher, args=('Streamlit', streamlit_process.stdout))
    monitor_thread.daemon = True
    monitor_thread.start()

    # Refactored URL Fetching Logic
    current_tunnel_url = None
    if IN_COLAB:
        print("\n✅ 伺服器已在背景啟動 (Streamlit process potentially running).")
        print("   ⏳ 建議等待約 15-30 秒讓 Streamlit 服務完成初始化...")
        try:
            print("   ➡️ 初始化完成後，請按 Enter 鍵 (或等待幾秒) 以嘗試獲取應用程式網址...")
            time.sleep(5) 
            print("   繼續嘗試獲取 URL...")
        except KeyboardInterrupt:
            print("\n⌨️ 操作被使用者中斷。")
            raise 
        except Exception as e:
            print(f"\n⚠️ 等待或輸入時發生錯誤: {e}。繼續嘗試獲取URL。")
        
        current_tunnel_url = fetch_url_with_eval_js(PORT, EVAL_JS_MAX_RETRIES, EVAL_JS_RETRY_DELAY, eval_js, time, print, IN_COLAB)
        if not current_tunnel_url:
             print("\n   ➡️ eval_js method failed. Will try cloudflared as a fallback if necessary.")

    if not current_tunnel_url:
        current_tunnel_url = fetch_url_with_cloudflared(PORT, CLOUDFLARED_TIMEOUT_SECONDS, subprocess, os, threading, time, print)

    display_result_html(current_tunnel_url, SUCCESS_HTML_TEMPLATE_STR, FAILURE_HTML_TEMPLATE_STR, display, HTML, clear_output, IN_COLAB, print)

    print("\n--- 應用程式即時輸出 (包含日誌與錯誤) ---")
    monitor_thread.join()

except KeyboardInterrupt:
    print("\n\n⌨️ 偵測到手動中斷。")
except Exception as e:
    print(f"\n\n❌ 腳本執行期間發生錯誤: {e}")
    import traceback
    traceback.print_exc() 
finally:
    print("\n\n--- 正在終止 Streamlit 服務 ---")
    if streamlit_process and streamlit_process.poll() is None:
        streamlit_process.terminate()
        try:
            streamlit_process.wait(timeout=10) 
            print("    Streamlit 程序已終止。")
        except subprocess.TimeoutExpired:
            print("    Streamlit 程序未能優雅終止，強制結束。")
            streamlit_process.kill()
    
    if monitor_thread and monitor_thread.is_alive():
        monitor_thread.join(timeout=5) 
    print("--- 腳本執行完畢 ---")
